# Background correction

The illumination in fluorescence imaging can be uneven due to for example 
the profile of the laser beam. Other source of background signal is the 
scattering of light which results in a sample dependant background.


In [2]:
import tifffile
import numpy as np

filename = '../data/example.tif'
img = tifffile.imread(filename)
print("The shape of the array is [depth x channels x height x width]", img.shape)


FileNotFoundError: [Errno 2] No such file or directory: '/lmb/home/data/example.tif'

Let's apply a 3D gaussian filter to the image, channel by channel:

In [ ]:
from skimage import filters
import napari

# Let's smooth each channels
smoothed = np.ones(img.shape)
for channel in range(img.shape[1]):
    smoothed[:,channel,:,:] = filters.gaussian(np.squeeze(img[:,channel,:,:]), 10)

# Let's visualize a single 2D plane
plt.imshow(smoothed[10,0,:,:])

# create a Viewer and add an image here with the two channels and scaling
napari.view_image(smoothed, channel_axis=1, name=['dapi', 'fitc'], scale=[10,1,1])


## Background substraction using the rolling ball algorithm
The rolling ball algorithm was introduced in 

Sternberg S. Biomedical Image Processing, IEEE Computer 1983;16(1):22-34. (DOI:10.1109/MC.1983.1654163)

This approach is not ideal for quantitative measurement of intensities but can 
help segment the image. 

We use here the implementation provided in the scikit-image package and the
 example is inspired from https://scikit-image.org/docs/stable/auto_examples/segmentation/plot_rolling_ball.html

First, let's define first a function to help display the image, background and corrected image. The histogram on the right of the figure helps understand how background correction affect the distribution of intensities.

In [ ]:
import matplotlib.pyplot as plt
from skimage.exposure import histogram
import numpy as np

def plt_bg_corr(im,bg,corrected):
    ''' Display background correction results '''
    fig, ax = plt.subplots(1, 4)
    ax[0].imshow(im,cmap='gray')
    ax[0].set_title('Image')
    ax[0].set_axis_off()
    ax[1].imshow(bg,cmap='gray')
    ax[1].set_title('Background')
    ax[1].set_axis_off()
    ax[2].imshow(corrected,cmap='gray')
    ax[2].set_title('Corrected')
    ax[2].set_axis_off()
    hist1, hist_centers1  = histogram(im)
    ax[3].loglog(hist_centers1, hist1, lw=2, label='before')
    hist2, hist_centers2  = histogram(corrected)
    ax[3].loglog(hist_centers2, hist2, lw=2,label='after')
    ax[3].set_title('Histogram')
    ax[3].set_position([0.8, 0.4, 0.2, 0.25])
    ax[3].legend()
    ax[3].set_xlabel('intensity (log)')


Let's load a test image too

In [ ]:
from skimage import io
import numpy as np
im = io.imread('../data/nuclei.tif')
im = im[0:500,0:500]
N,M = im.shape
Y,X = np.ix_(np.arange(N),np.arange(M))
# We add a bias defined as X/5 and convert the elements of the array to 16 bit unsiged integers.
im = np.asarray(im + X/5, dtype=np.uint16)
plt.imshow(im)
plt.axis('off')
plt.show()

Now we an apply a rolling ball filter to estimate the background:

In [ ]:
from skimage import restoration
from timeit import default_timer as timer

tic = timer()

# Apply a rolling ball to estimate a background image
bg = restoration.rolling_ball(im, radius=60)

# Substract the background to the image 
corrected = im - bg

# Display the image, the estimated background and the corrected image
plt_bg_corr(im,bg,corrected)

# Display elapsed time for this step
etime = timer() - tic
print(f"Elapsed time: {etime:.2f} s")


We might be unhappy with the results as the intensity and spatial dimension of the rolling ball are here linked. To decouple them, we can specify a kernel:

In [ ]:
from skimage import restoration
from timeit import default_timer as timer

tic = timer()

# define a kernel for the rolling ball with its shape and intensity
kernel = restoration.ellipsoid_kernel((120,120),  10)

# Apply a rolling ball to estimate a background image
bg = restoration.rolling_ball(im, kernel=kernel)

# Substract the background to the image 
corrected = im - bg

# Display the image, the estimated background and the corrected image
plt_bg_corr(im,bg,corrected)

# Display elapsed time for this step
etime = timer() - tic
print(f"Elapsed time: {etime:.2f} s")

## Background substraction using a top-hat
The top-hat transform is defined as the difference between the image and it's morphological opening. The morphological opening of an image can be defined as a minimum filter or morphological erosion followed by a maximum filter or morphological dilation.

In [ ]:
from skimage import morphology
from skimage.filters.rank import maximum, minimum
from skimage.filters import gaussian
from timeit import default_timer as timer

# We define a structuring element (flat kernel) with radius 30
selem =  morphology.disk(30)

tic = timer()
# We can either directly compute the corrected image
corrected = morphology.white_tophat(im, selem)

# and deduce the background.
bg = im - corrected

# Display the image, the estimated background and the corrected image
plt_bg_corr(im,bg,corrected)

# Display elapsed time for this step
etime = timer() - tic
print(f"Elapsed time: {etime:.2f} s")

# Let's test the same by coding directly the Top-Hat function
tic = timer()

# Define the erosion
bg2 = maximum(minimum(im, selem), selem)

# Eventually, we can post process the background 
bg2 = gaussian(np.asarray(bg2,dtype=float), 20)

# Compute the residuals to obtain the top-hat
corrected2 = np.asarray(im,dtype=float)  - bg2

# Display again the results and timing
plt_bg_corr(im,bg2,corrected2)
etime = timer() - tic
print(f"Elapsed time: {etime:.2f} s")
